<a href="https://colab.research.google.com/github/anamclernon/binary-images-classification/blob/main/Binary_Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Binary Image Classifier

In [ ]:
%tensorflow_version 2.x
%matplotlib inline 


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import os
from os.path import isfile
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import imghdr

In [ ]:
def load_imgs_from_directory(path_name, class_names, test_split=0.2, validation_split=0.2):
  dataset = tf.keras.utils.image_dataset_from_directory(path_name, labels="inferred", batch_size=None,
                                            class_names=class_names, image_size=(128, 128))
  images_normalized = [batch[0] /255 for batch in dataset]
  labels = [batch[1] for batch in dataset]
  size = len(labels)
  index_split = round(size * test_split)
  index_validation_split = size- round(size * validation_split)
  test_images = images_normalized[:index_split]
  test_labels = labels[:index_split]
  val_images = images_normalized[index_validation_split:]
  val_labels = labels[index_validation_split:]
  tr_images = images_normalized[index_split:index_validation_split]
  tr_labels = labels[index_split:index_validation_split]

  return np.array(tr_images), np.array(tr_labels), np.array(val_images), np.array(val_labels), np.array(test_images), np.array(test_labels)


In [ ]:
def create_model(learning_rate, metrics):

  model = tf.keras.models.Sequential()

  model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(128, 128, 3)))
  model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Flatten())
  model.add(layers.Dense(units=128, activation="relu"))
  model.add(layers.Dense(units=1, activation="sigmoid"))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=metrics)

  return model


In [ ]:
def train_model(model, train_images, train_labels, batch_size, epochs, validation_data):
  history = model.fit(x=train_images, y=train_labels, batch_size=batch_size, epochs=epochs, 
                      validation_data=validation_data, shuffle=True)
  
  epochs=history.epoch

  hist = pd.DataFrame(history.history)

  return epochs, hist



In [ ]:
def plot_curves(epochs_no, epochs2, hist2, list_metrics):
  plt.figure()
  plt.xlim([0, epochs_no])
  plt.ylim([0,1])
  plt.xlabel("Epoch")
  plt.ylabel("Value")
  for m in list_metrics:
    plt.plot(epochs2[1:], hist2[m][1:], label=m)
  plt.legend()
  plt.show()

In [ ]:
path_name = "/content/drive/MyDrive/Dataset/cowsheep/cowswaterfall"
class_names = ["cows", "waterfalls"]

In [ ]:
tr_images, tr_labels, val_images, val_labels, test_images, test_labels = load_imgs_from_directory(path_name, class_names, test_split=0.2)

Found 11538 files belonging to 2 classes.


In [ ]:
sum(tr_labels)/len(tr_labels)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(tr_images[69], interpolation='nearest')
plt.show()

In [ ]:
batch_size = 32
validation_split=0.2
learning_rate=0.1
epochs=200
list_metrics = [tf.keras.metrics.BinaryAccuracy(name="bin_acc")]

my_model = create_model(learning_rate, list_metrics)

In [ ]:
epochs2, hist2 = train_model(my_model, tr_images, tr_labels, batch_size, epochs, validation_data=(val_images, val_labels))


In [ ]:
plot_curves(epochs, epochs2, hist2, ["bin_acc", "val_bin_acc"])

In [ ]:
my_model.evaluate(x=test_images, y=test_labels, batch_size=16)